# Challenge 3

In this challenge we will work on the `Orders` data set. In your work you will apply the thinking process and workflow we showed you in Challenge 2.

You are serving as a Business Intelligence Analyst at the headquarter of an international fashion goods chain store. Your boss today asked you to do two things for her:

**First, identify two groups of customers from the data set.** The first group is **VIP Customers** whose **aggregated expenses** at your global chain stores are **above the 95th percentile** (aka. 0.95 quantile). The second group is **Preferred Customers** whose **aggregated expenses** are **between the 75th and 95th percentile**.

**Second, identify which country has the most of your VIP customers, and which country has the most of your VIP+Preferred Customers combined.**

## Q1: How to identify VIP & Preferred Customers?

We start by importing all the required libraries:

In [3]:
# import required libraries
import numpy as np
import pandas as pd

Next, extract and import `Orders` dataset into a dataframe variable called `orders`. Print the head of `orders` to overview the data:

In [5]:
# your code here

orders = pd.read_csv('Orders.csv')
orders.head(5)

,Unnamed: 0,InvoiceNo,StockCode,year,month,day,hour,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,amount_spent
0,0,536365,85123A,2010,12,3,8,white hanging heart t-light holder,6,12/1/2010 8:26,2.55,17850,United Kingdom,15.30
1,1,536365,71053,2010,12,3,8,white metal lantern,6,12/1/2010 8:26,3.39,17850,United Kingdom,20.34
2,2,536365,84406B,2010,12,3,8,cream cupid hearts coat hanger,8,12/1/2010 8:26,2.75,17850,United Kingdom,22.00
3,3,536365,84029G,2010,12,3,8,knitted union flag hot water bottle,6,12/1/2010 8:26,3.39,17850,United Kingdom,20.34
4,4,536365,84029E,2010,12,3,8,red woolly hottie white heart.,6,12/1/2010 8:26,3.39,17850,United Kingdom,20.34


---

"Identify VIP and Preferred Customers" is the non-technical goal of your boss. You need to translate that goal into technical languages that data analysts use:

## How to label customers whose aggregated `amount_spent` is in a given quantile range?


We break down the main problem into several sub problems:

#### Sub Problem 1: How to aggregate the  `amount_spent` for unique customers?

#### Sub Problem 2: How to select customers whose aggregated `amount_spent` is in a given quantile range?

#### Sub Problem 3: How to label selected customers as "VIP" or "Preferred"?

*Note: If you want to break down the main problem in a different way, please feel free to revise the sub problems above.*

Now in the workspace below, tackle each of the sub problems using the iterative problem solving workflow. Insert cells as necessary to write your codes and explain your steps.

In [8]:
# your code here

spent_per_customer = orders.groupby(['CustomerID', 'Country'], as_index = False).agg({'amount_spent':'sum'})
spent_per_customer

,CustomerID,Country,amount_spent
0,12346,United Kingdom,77183.60
1,12347,Iceland,4310.00
2,12348,Finland,1797.24
3,12349,Italy,1757.55
4,12350,Norway,334.40
...,...,...,...
4342,18280,United Kingdom,180.60
4343,18281,United Kingdom,80.82
4344,18282,United Kingdom,178.05
4345,18283,United Kingdom,2094.88


Now we'll leave it to you to solve Q2 & Q3, which you can leverage from your solution for Q1:

## Q2: How to identify which country has the most VIP Customers?

In [11]:
# your code here

vip = spent_per_customer[spent_per_customer['amount_spent'] > spent_per_customer['amount_spent'].quantile(.95)]
preferred = spent_per_customer[(spent_per_customer['amount_spent'].quantile(.75) < spent_per_customer['amount_spent']) & (spent_per_customer['amount_spent'] < spent_per_customer['amount_spent'].quantile(.95))]

In [12]:
preferred

,CustomerID,Country,amount_spent
1,12347,Iceland,4310.00
2,12348,Finland,1797.24
3,12349,Italy,1757.55
5,12352,Norway,2506.04
9,12356,Portugal,2811.43
...,...,...,...
4327,18259,United Kingdom,2338.60
4328,18260,United Kingdom,2643.20
4336,18272,United Kingdom,3078.58
4345,18283,United Kingdom,2094.88


## Q3: How to identify which country has the most VIP+Preferred Customers combined?

In [13]:
# your code here

vip['Country'].value_counts()

United Kingdom     178
Germany             11
France               9
Switzerland          3
Spain                2
Japan                2
Portugal             2
EIRE                 2
Australia            1
Finland              1
Norway               1
Denmark              1
Cyprus               1
Singapore            1
Netherlands          1
Channel Islands      1
Sweden               1
Name: Country, dtype: int64

In [14]:
combined = pd.concat([vip, preferred])
combined['Country'].value_counts()

United Kingdom     934
Germany             39
France              29
Belgium             11
Switzerland          9
Norway               7
Spain                7
Portugal             7
Finland              5
Italy                5
Australia            4
Channel Islands      4
Japan                4
EIRE                 3
Denmark              3
Cyprus               3
Sweden               2
Israel               2
Netherlands          1
Singapore            1
Iceland              1
Austria              1
Lebanon              1
Poland               1
Greece               1
Canada               1
Malta                1
Name: Country, dtype: int64